In [1]:
from circuits import trotter_2D_square_inhomogeneous as circ
import numpy as np
from matplotlib import pyplot as plt
from qiskit import transpile
import quimb as qu
import algo_cooling as algo
import register_ as reg
import quimb.tensor as qtn
from tqdm import tqdm 
import quf
import time
from floquet.circuits import (trotter_2D_square, circ_gates, Z1_ti, Z2_ti, mpo_z2_, simulate, transpile)
import math
import algo_dmrg


In [2]:
import torch
to_backend = algo.backend_torch(device = "cuda", dtype = torch.complex128)
to_backend_ = algo.backend_numpy( dtype = "complex128")

opt = algo.opt_(progbar=False)

In [3]:
Lx, Ly = 3, 4  # Lx < Ly
L = Lx * Ly
steps = 22
gate_info, depth_map, circ_info = quf.circ_info(Lx=Lx, Ly=Ly, steps=steps)

In [4]:
z2_exact = qu.load_from_disk(f"z2_exact/z2_ext_Lx{Lx}Ly{Ly}")
# z2_exact

In [5]:
res_dmrg = {}
res_dmrg = qu.load_from_disk("store/res_dmrg")
res_dmrg.setdefault(f"L_{L}", [])
res_dmrg.keys()

dict_keys(['L_12', 'F_L12_chi16', 'Scale_L12_chi16', 'Z2_L12_chi16', 'Z2_L12', 'Norm_L12_chi16', 'Gamma_L12_chi16', 'F_L12_chi32', 'Scale_L12_chi32', 'Z2_L12_chi32', 'Norm_L12_chi32', 'Gamma_L12_chi32'])

In [6]:
mpoz2 = mpo_z2_(circ_info, "Z", where_=L//2, form="left")
mpoz2.apply_to_arrays(to_backend)
mpoz2.show()

│2│2│2│2│2│2│2│1│1│1│1│
>─>─>─>─>─>─>─>─>─>─>─●
│ │ │ │ │ │ │ │ │ │ │ │


In [7]:
# prep state and bond dimenion
chi = 8

p = qtn.MPS_computational_state([0]*L)
p.expand_bond_dimension(chi)
info_c = {}
p.canonicalize_([L//2], cur_orthog='calc', info = info_c)
cur_orthog = [L//2, L//2] 

p.apply_to_arrays(to_backend)


In [8]:
p.show()

 2 4 8 8 8 8 8 8 8 4 2 
>─>─>─>─>─>─●─<─<─<─<─<
│ │ │ │ │ │ │ │ │ │ │ │


In [9]:
%%time
F_l, Fidel_l, Norm_l, norm, tqg_, f = ([], [1], [1], 1, 0, 1)

e_x_ = {}
gamma = [] 
res = {f"F_L{L}_chi{chi}":[], f"Scale_L{L}_chi{chi}":[], f"Z2_L{L}_chi{chi}":[], f"Z2_L{L}":z2_exact}
res |= {f"F_L{L}_chi{chi}":[], f"Norm_L{L}_chi{chi}":[], f"Gamma_L{L}_chi{chi}":[]}



with tqdm(total=len(gate_info),  desc="dmrg:", leave=True, position=0, 
        colour='CYAN', disable = not True) as pbar:

    for (where, count), G in gate_info.items():

        if len(where) == 1:
            algo_dmrg.gate_1d(p, where, to_backend(G), inplace=True)

        if len(where) == 2:
            tqg_ += 1
            # Canonize around where using cur_orthog
            algo_dmrg.canonize_mps(p, where, cur_orthog)

            # build target tensor-network
            p_g = algo_dmrg.gate_1d(p, where, to_backend(G), contract='split-gate', inplace=False)

            # run dmrg
            fit = algo_dmrg.FIT(p_g, p=p, opt=opt, re_tag=False, range_int=cur_orthog)
            fit.run_gate(n_iter=6, verbose=False)
            #fit.run_eff(n_iter=15, verbose=False)
            p = fit.p

            # get un-normalized fidelity
            # norm = p.norm()
            Fidel_l.append( complex(fit.loss_[-1] ** 2).real )


        
        pbar.set_postfix({"tqg_":tqg_, "F": Fidel_l[-1]})
        pbar.refresh()
        pbar.update(1)

        # measure an observable
        if count+1 in list(depth_map.keys()):
            depth = depth_map[count+1]
            # print("depth", depth, "count", count+1)
            e_x = algo_dmrg.energy_global(mpoz2, p, opt=opt)
            e_x = complex(e_x).real
            e_x_[depth] = complex(e_x).real

            res[f"F_L{L}_chi{chi}"].append( Fidel_l[-1] )
            res[f"Scale_L{L}_chi{chi}"].append( z2_exact[depth]/ e_x )
            res[f"Norm_L{L}_chi{chi}"].append( complex(p.norm()).real )
            res[f"Z2_L{L}_chi{chi}"].append( e_x )
            res[f"Gamma_L{L}_chi{chi}"].append( np.log(z2_exact[depth]/ e_x) / (np.log(Fidel_l[-1])+1.e-12) )
            

dmrg:: 100%|█████████████████████████████████████████████████████████████████████| 1116/1116 [00:38<00:00, 29.23it/s, tqg_=528, F=0.22]

CPU times: user 37.3 s, sys: 594 ms, total: 37.9 s
Wall time: 38.2 s


In [10]:
e_x = algo_dmrg.energy_global(mpoz2, p, opt=opt)


In [11]:
complex(Fidel_l[-1]).real

0.21958934904985997

In [12]:
p_dmrg = p.copy()
p_dmrg.norm()

tensor(0.4686-2.5641e-18j, device='cuda:0', dtype=torch.complex128)

In [13]:
res_dmrg |= res
print(res_dmrg.keys())
res_dmrg[f"L_{L}"].append(chi)
qu.save_to_disk(res_dmrg, "store/res_dmrg")

dict_keys(['L_12', 'F_L12_chi16', 'Scale_L12_chi16', 'Z2_L12_chi16', 'Z2_L12', 'Norm_L12_chi16', 'Gamma_L12_chi16', 'F_L12_chi32', 'Scale_L12_chi32', 'Z2_L12_chi32', 'Norm_L12_chi32', 'Gamma_L12_chi32', 'F_L12_chi8', 'Scale_L12_chi8', 'Z2_L12_chi8', 'Norm_L12_chi8', 'Gamma_L12_chi8'])


['store/res_dmrg']

In [14]:
res_dmrg[f"L_{L}"]

[16, 32, 8]

In [15]:
# res_dmrg

In [16]:
s

NameError: name 's' is not defined

In [ ]:
%%time
p = qtn.MPS_computational_state([0]*L)
info_c = {}
p.canonicalize_([L//2], cur_orthog='calc', info=info_c)
p.apply_to_arrays(to_backend)


chi = 64

F_l = []
tqg_ = 0
norm_1 = 1
Fidel_l = [1]
f = 1
res_ = {f"F_L{L}_chi{chi}":[], f"Gamma_L{L}_chi{chi}":[], f"Z2_L{L}_chi{chi}":[], f"Z2_L{L}":z2_exact}
res_ |= {f"F_L{L}_chi{chi}":[], f"Norm_L{L}_chi{chi}":[], }

with tqdm(total=len(gate_info),  desc="svd:", leave=True, position=0, 
        colour='CYAN', disable = not True) as pbar:

    for (where, count), G in gate_info.items():

        if len(where)==1:
            p.gate_(to_backend(G), where, contract=True, inplace=True)
        else:
            tqg_ += 1
            
            #norm_0 = p.norm()
            p_g = p.gate(to_backend(G), where, inplace=False, contract=False)
            
            p.gate_nonlocal_(to_backend(G), where, max_bond = chi , info=info_c , method='direct', **{"cutoff":1.e-12})
            norm = p.norm()
            

            f = algo_dmrg.fidel_mps(p, p_g)
            F_l.append( complex(f).real  )
            
            # measure Fidelity
            Fidel = 0
            for f_ in F_l:
                Fidel += np.log(f_)
            Fidel_l.append( np.exp(Fidel) )

        # measure an observable
        if count+1 in list(depth_map.keys()):
            depth = depth_map[count+1]
            print("depth", depth, "count", count+1)
            e_x = algo_dmrg.energy_global(mpoz2, p, opt=opt)
            e_x = complex(e_x).real
            e_x_[depth] = complex(e_x).real

            res_[f"F_L{L}_chi{chi}"].append( Fidel_l[-1] )
            res_[f"Gamma_L{L}_chi{chi}"].append( z2_exact[depth]/ e_x )
            res_[f"Norm_L{L}_chi{chi}"].append( complex(p.norm()).real )
            res_[f"Z2_L{L}_chi{chi}"].append( e_x )
        pbar.set_postfix({"tqg_":tqg_, "Fidel":Fidel_l[-1], "bnd":p.max_bond() , "Norm": round(complex(norm).real,4)})
        pbar.refresh()
        pbar.update(1)



In [ ]:
p_svd = p.copy()
p_svd.norm()

In [ ]:
complex(Fidel_l[-1]), complex(p_svd.norm())

In [ ]:
f = algo_dmrg.fidel_mps(p_svd, p_dmrg)
f

In [ ]:
e_x = algo_dmrg.energy_global(mpoz2, p, opt=opt)
e_x

In [ ]:
# e_x_
# import matplotlib.pyplot as plt

# # suppose e_x_ is a 1D array or list of numbers
# # e.g.
# # e_x_ = [0.1, 0.2, 0.25, 0.3, 0.28, 0.26]

# plt.plot([i for i in range(Lx)],list(e_x_[1]), marker='o', linestyle='-', color='b')
# plt.title("Evolution of e_x_")
# plt.xlabel("Step")
# plt.ylabel("e_x_ value")
# plt.grid(True)
# plt.show()


In [ ]:
res_